In [1]:
import cv2
import numpy as np
import os

In [2]:
MAIN_IMAGE_PATH = 'image/img2.jpg'
CROP_IMAGE_PATH = 'croped/img2_croped.jpg'

def get_chromaticity(img):
    img_float = img.astype(np.float32)
    B, G, R = cv2.split(img_float)
    intensity = R + G + B
    intensity[intensity < 1e-5] = 1.0
    x = R / intensity
    y = G / intensity
    return x, y

In [3]:
def train_skin_model_from_crop(crop_img):
    x, y = get_chromaticity(crop_img)
    x_flat = x.flatten()
    y_flat = y.flatten()
    data = np.vstack((x_flat, y_flat))
    mean_vector = np.mean(data, axis=1)
    covariance_matrix = np.cov(data)
    return mean_vector, covariance_matrix

In [4]:
def detect_skin(img, mean, cov):
    x, y = get_chromaticity(img)
    h, w = x.shape
    pixel_data = np.dstack((x, y)).reshape(-1, 2)
    inv_cov = np.linalg.inv(cov)
    diff = pixel_data - mean
    dist = np.sqrt(np.sum(np.dot(diff, inv_cov) * diff, axis=1))
    dist_map = dist.reshape(h, w)
    
    # Threshold < 2.5 covers approx 98% of the skin distribution
    mask = dist_map < 2.5
    return mask.astype(np.uint8) * 255



In [5]:
print(123)

123


In [6]:
base_name = os.path.splitext(os.path.basename(MAIN_IMAGE_PATH))[0]
output_filename = f"{base_name}_results.jpg"

print(f"Processing: {MAIN_IMAGE_PATH}")
print(f"Target Output: {output_filename}")

main_img = cv2.imread(MAIN_IMAGE_PATH)
skin_crop = cv2.imread(CROP_IMAGE_PATH)

if main_img is None:
    print(f"Error: Could not load {MAIN_IMAGE_PATH}")
elif skin_crop is None:
    print(f"Error: Could not load {CROP_IMAGE_PATH}")
else:
    # 2. Train and Detect
    print("Training model...")
    mean, cov = train_skin_model_from_crop(skin_crop)
    
    print("Detecting skin...")
    final_result_image = detect_skin(main_img, mean, cov)
    
    # 3. Save to root directory
    cv2.imwrite(output_filename, final_result_image)
    print(f"SUCCESS: Saved as '{output_filename}'")
    
    # 4. Show
    cv2.imshow("Original", main_img)
    cv2.imshow("Result (White=Skin)", final_result_image)
    
    print("Press any key to close windows.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Processing: image/img2.jpg
Target Output: img2_results.jpg
Training model...
Detecting skin...
SUCCESS: Saved as 'img2_results.jpg'
Press any key to close windows.
